# Simulación del Protocolo BB84
Esta notebook simula el protocolo BB84 entre Alice y Bob, incluyendo la opción de un espía (Eve).

In [7]:
import random
import pandas as pd

# Función para elegir una base aleatoria: recta (|) o diagonal (/)
def random_base():
    return random.choice(["|", "/"])

# Función para codificar un bit en base
def encode_bit(bit, base):
    # En base recta (|): 0 => ↕, 1 => ↔
    # En base diagonal (/): 0 => ↖, 1 => ↘
    if base == "|":
        return "↕" if bit == 0 else "↔"
    else:
        return "↖" if bit == 0 else "↘"

# Simulación de n rondas de transmisión
def simulate_bb84(n=15, include_eve=False):
    rounds = []
    for i in range(n):
        bit_alice = random.randint(0, 1)
        base_alice = random_base()
        photon = encode_bit(bit_alice, base_alice)

        base_bob = random_base()

        # Bob mide el fotón
        if base_bob == base_alice:
            bit_bob = bit_alice
            match = "✔"
            use = "✅"
        else:
            bit_bob = random.randint(0, 1)
            match = "✘"
            use = "❌"

        # Intercepción por parte de Eve
        if include_eve:
            base_eve = random_base()
            if base_eve == base_alice:
                bit_eve = bit_alice
            else:
                bit_eve = random.randint(0, 1)
            # Photon puede ser alterado por Eve
            photon = encode_bit(bit_eve, base_eve)
            # Rehacer medición de Bob
            if base_bob == base_eve:
                bit_bob = bit_eve
            else:
                bit_bob = random.randint(0, 1)

        rounds.append({
            "Ronda": i + 1,
            "Bit de Alice": bit_alice,
            "Base de Alice": base_alice,
            "Fotón enviado": photon,
            "Base de Bob": base_bob,
            "Bit de Bob": bit_bob,
            "¿Coinciden bases?": match,
            "¿Usar bit?": use,
        })

    df = pd.DataFrame(rounds)
    print(df)
    
    # Generar clave secreta
    secret_key = "".join(
        str(row["Bit de Bob"]) for _, row in df.iterrows() if row["¿Usar bit?"] == "✅"
    )
    print(f"\nClave secreta compartida: {secret_key}")
    return df, secret_key

# Ejecutar la simulación
simulate_bb84(n=20, include_eve=True)


    Ronda  Bit de Alice Base de Alice Fotón enviado Base de Bob  Bit de Bob  \
0       1             1             /             ↘           /           1   
1       2             0             |             ↕           |           0   
2       3             1             /             ↘           /           1   
3       4             0             /             ↖           |           0   
4       5             1             /             ↘           /           1   
5       6             0             /             ↖           /           0   
6       7             0             /             ↔           /           1   
7       8             0             /             ↖           |           0   
8       9             0             /             ↔           /           0   
9      10             1             /             ↘           |           1   
10     11             0             /             ↖           |           0   
11     12             0             |             ↕ 

(    Ronda  Bit de Alice Base de Alice Fotón enviado Base de Bob  Bit de Bob  \
 0       1             1             /             ↘           /           1   
 1       2             0             |             ↕           |           0   
 2       3             1             /             ↘           /           1   
 3       4             0             /             ↖           |           0   
 4       5             1             /             ↘           /           1   
 5       6             0             /             ↖           /           0   
 6       7             0             /             ↔           /           1   
 7       8             0             /             ↖           |           0   
 8       9             0             /             ↔           /           0   
 9      10             1             /             ↘           |           1   
 10     11             0             /             ↖           |           0   
 11     12             0             |  